# The Setup

Our first step in the journey is reading and cleaning the data. We won't focus too much on this step because the really interesting stuff happens once we have the data loaded. However, we will showcase how the data needs to be formatted for Gurobi. If this is your first pass of the case study, feel free to skip over this section and return to it later when you want a more in-depth overview of formatting the data.

Before we begin, I wanted to make a quick note about a line of code you'll see repeated throughout the notebook: %%script false --no-raise-error


- **NOTE:** This is just cell magic (if you're unfamiliar, think of it as using the cmd line) telling the notebook not to run this cell. We'll use it in various places to demonstrate ideas or code snippets that are not meant to produce an output

## The Environment

The fun always starts with importing packages. 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import gurobipy as gp
from gurobipy import GRB

The next and final step in this section is to define what data we'll be using to implement the solver, and in this case, we'll be using real-world data from Madagascar. Madagascar is the fourth largest island in the world and is located off the southeastern coast of Africa. Known for its unique biodiversity, approximately 90% of its wildlife is found nowhere else on Earth. The island's diverse ecosystems range from rainforests to deserts, making it a hotspot for biological research and conservation efforts.

However, Madagascar is also prone to natural disasters, including cyclones, floods, and droughts, all of which have a significant impact on its population and infrastructure. These disasters pose challenges for disaster response and resource allocation, making it an ideal case study for optimization and data-driven decision-making.

## Reading and Cleaning 
Now we'll load the data for a set of natural disasters and warehouses that will store supplies. One assumption made in this case study is that we do not have to worry about warehouse capacity.

<div class="alert alert-warning">
    <strong>Note!</strong>
    <p>While the data shows latitude and longitude of warehouse locations, these coordinates have been altered slightly.</p>
</div>

In [2]:
disasters = pd.read_csv("data/disasters.csv")
disasters.head()

,Type,Lat,Long,People Impacted
0,Storm,-12.2667,49.2833,118000
1,Storm,-14.2667,50.1667,100215
2,Epidemic,-14.8762,47.9835,21976
3,Epidemic,-15.7167,46.3167,15172
4,Flood,-16.9504,46.8281,20000


In [3]:
warehouses = pd.read_csv("data/warehouses.csv")
warehouses.head()

,Type,Lat,Long,Buckets
0,Warehouses,-17.8237,48.4263,26
1,Warehouses,-20.5167,47.2500,41
2,Warehouses,-18.9085,47.5375,9046
3,Warehouses,-17.3843,49.4098,2762
4,Warehouses,-16.9167,49.9000,1682


## Data Exploration and Visualization

It's always useful to see the general structure of data and the broader context of the problem. Now that we've cleaned and loaded it, let's take some time to understand it. The goal here should be to get comfortable with the problem as a whole and give you an intuitive understanding of what we are solving. So, the emphasis isn't on the code. For this reason, most of the following cells have been written as functions to collapse more easily across different platforms. Your goal shouldn't be to understand the libraries used to map, but instead on how Madagascar looks at a high level.

### Where is Everything?
One of the first things we'll look at is where the supplies are relative to disasters right now. 

![](images/book2_map1.png)

If you are working with an interactive version of this notebook, here is the code to create the above Plotly map. 

In [ ]:
# don't forget to remove the ''' to run this code!
'''
dfw = warehouses[['Type', 'Lat', 'Long']]
dfd = disasters[['Type', 'Lat', 'Long']]

df_combined = pd.concat([dfw[['Lat', 'Long', 'Type']], dfd[['Lat', 'Long', 'Type']]], ignore_index=True)

fig = px.scatter_mapbox(df_combined, lat="Lat", lon="Long", 
                        color="Type",
                        color_discrete_sequence=px.colors.qualitative.Safe,
                        zoom=4.5)

# Use a minimalist map style to reduce clutter
fig.update_layout(mapbox_style="open-street-map")
fig.update_traces(marker_size=10)

# Optionally center the map around the mean latitude and longitude of your points
mean_lat = df_combined['Lat'].mean()
mean_long = df_combined['Long'].mean()
fig.update_layout(mapbox_center={"lat": mean_lat, "lon": mean_long})
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.show()
'''

We can see that the warehouses are aligned closely with the most prominent disaster sights*, so assuming they are built to a level that can survive and protect against those disasters, Madagascar should be in a strong position in terms of coverage. However, there's more than meets the eye. Let's dive a little deeper!

Hopefully, now that you've gotten a sense of the country and the layout, we'll turn off some of the more detailed parts (such as roads and urbanization) so it's simpler to see what’s going on.

- **NOTE:** locations of warehouses have been modified for this case study for safety.

### What Supplies are Available?

An important question to ask at this point is: what do we have inside the warehouses? It's great that there seems to be good coverage that ensures a warehouse is nearby for any disaster, but if one of them is full of kitchen sets and there's a flood, they might not be as immediately useful as buckets or tarpaulins. So, let's look at the overall breakdown of supplies by warehouse/location:

![](images/barchart.png)

As we can see from above, the supplies skew heavily towards buckets, water containers, and mosquito nets, which makes sense for an island nation. But while it's great that we have a lot of buckets, it won't do us too much good if none of them are at the coast, for instance.

- **NOTE:** From here out in the case study, we're going to focus on just buckets as they're the most prevalent item and, for the purposes of this case, it's time consuming and repetitive to analyze all 15 item types of supplies available. 

This case study assumes that it is sufficient to analyze each item independently. 

### Where are the Supplies?

Let's take a look at the following map to get a better picture of where everything is.

In [5]:
supplies = pd.read_csv("data/supplies.csv")
supplies.head()

,Lat,Long,Buckets
0,-13.680400,48.455500,375
1,-17.823700,48.426300,26
2,-20.516700,47.250000,41
3,-25.176133,46.089378,2322
4,-18.908500,47.537500,9046


![](images/book2_map2.png)

Here's the code for this interactive map. 

In [6]:
# don't forget to remove the ''' to run this code!
#
#fig = px.scatter_mapbox(supplies, lat="Lat", lon="Long", size="Buckets",
#                       color_discrete_sequence=px.colors.qualitative.Safe,
#                       zoom=4.5,  # Adjust zoom level as needed
#                       )
## Optionally center the map around the mean latitude and longitude of your points
## Use a minimalist map style to reduce clutter
#fig.update_layout(mapbox_style="carto-positron")
#
#mean_lat = supplies['Lat'].mean()
#mean_long = supplies['Long'].mean()
#fig.update_layout(mapbox_center={"lat": mean_lat, "lon": mean_long})
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#
#fig.show()

We can see that the supplies are largely located on the eastern coast line of the country. 

### Not All Disasters Are Built the Same

Let's see which events Madagascar is more likely to encounter. 

![](images/book2_map3.png)

And the code, of course.

In [7]:
# don't forget to remove the ''' to run this code!
#
#fig = px.scatter_mapbox(disasters, lat="Lat", lon="Long", color="Type", 
#                        size="People Impacted",
#                        zoom=4.5,  # Adjust zoom level as needed
#                        color_discrete_sequence=px.colors.qualitative.Safe,
#                        )
#
## Optionally center the map around the mean latitude and longitude of your points
## Use a minimalist map style to reduce clutter
#fig.update_layout(mapbox_style="carto-positron")
#
#mean_lat = disasters['Lat'].mean()
#mean_long = disasters['Long'].mean()
#fig.update_layout(mapbox_center={"lat": mean_lat, "lon": mean_long})
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#
#fig.show()

### How Disasters and Supplies Compare in Scale

You can see small blue circles within the red circles, representing supply quantities. The size of each red circle indicates the estimated number of people needing supplies in a disaster-affected area. Meanwhile, the blue circles show the total number of items available, adjusted by how many people each item can serve. For example, one large bucket is estimated to meet the needs of 2.5 people, so the blue circles display the quantity of supplies as $2.5 \cdot Supplies$


![](images/book2_map4.png)

In [8]:
# don't forget to remove the ''' to run this code!
#
#dfs = supplies.copy()
#dfs['Type'] = 'Supplies'
#dfs['Scale'] = dfs['Buckets']*2.5
#
#dfd = disasters.copy()
#dfd['Scale'] = dfd['People Impacted']
#
#df_combined = pd.concat([dfs[['Type','Lat', 'Long', 'Scale']],
#                         dfd[['Type','Lat', 'Long', 'Scale']]],
#                         ignore_index=True)
#
#fig = px.scatter_mapbox(df_combined, lat="Lat", lon="Long", 
#                      size="Scale", color='Type',
#                      size_max=40,  # Maximum size of the marker
#                      zoom=4.5,  # Adjust zoom level as needed
#                      color_discrete_sequence=px.colors.qualitative.Safe,
#                      )
#
## Optionally center the map around the mean latitude and longitude of your points
## Use a minimalist map style to reduce clutter
#fig.update_layout(mapbox_style="carto-positron")
#
#mean_lat = df_combined['Lat'].mean()
#mean_long = df_combined['Long'].mean()
#fig.update_layout(mapbox_center={"lat": mean_lat, "lon": mean_long})
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#
#fig.show()


### How Does ESUPS Communicate this?


Effective communication is vital in disaster response, not only for coordinating logistics but also for ensuring that the right information reaches the right people at the right time. In this section, we will explore the ESUPS website, which serves as a central hub for sharing critical data and insights with a diverse audience, including NGOs, government agencies, and the public.

The ESUPS STOCKHOLM platform is designed to translate complex optimization models and logistical data into intuitive, actionable information. Through a combination of interactive charts, maps, and data visualizations, the platform provides users (comprised of NGOs and humanitarian/relief organizations) with a clear understanding of where pre-positioned supplies are located, how they can be accessed, and the impact of various logistical decisions.

We will examine how ESUPS effectively communicates information about stock and analysis of results through STOCKHOLM, highlighting key features such as:
•	Real-time Data Visualizations: How ESUPS uses dynamic maps and graphs to show the distribution of resources and potential gaps in coverage. These are updated daily with the newest information available
•	User-Friendly Interface: The strategies employed to make the website accessible and informative for both technical and non-technical users.
•	Impact Metrics: How ESUPS showcases the tangible benefits of its optimization strategies, including improved response times and resource utilization. We'll take a special look at these later in the case to help us understand what our optimization techniques have achieved.

And finally, in one of the most relevant features of this case study, ESUPS has given us permission to share some of the difficulties they have experienced in their attempts to convey the scale of this problem and their findings along with what they hope to change and improve for their users going forward.

Let's take a look!

ESUPS's platform Stockholm, which is their platform for hosting this model (you can apply for access by visiting the homepage linked above), has two main pages, the first is the context page, and explains the context of the problem that we've been discussing in the past few sections, along with additional details about their mission and collaborating organizations. We'll look at this one first!

**The Real-time Data Visualization** : Progessive Improvement

We're going to take a slight detour here to look at how all this information is communicated in real life! You can skip this section without concern for missing important information. 

As we've seen, getting an intuitive understanding of disasters and humanitarian supplies can be tricky when working with a new country. In fact, it's one of the main challenges ESUPS has faced in driving adoption and explaining why their work is so important. 

Data Visualization isn't an easy task, but let's take a look at how ESUPS started and how they're continuing to improve!

![](images/Old_Stockholm.png)

The earlier version of the ESUPS dashboard, while not perfect, played a critical role in enabling the organization to quickly deliver a functional product that demonstrated tangible results to other non-profits. This initial version provided the essential tools needed to map stockpiles and showcase the value of the platform in real-world scenarios. By prioritizing functionality and speed, ESUPS was able to meet the immediate needs of its partners, proving the concept and gaining crucial buy-in from stakeholders.

The design, though basic, allowed for rapid deployment and iteration, proving that in the early stages of development, it's more important to have a working solution than to strive for perfection. The dashboard's ability to deliver key insights swiftly and effectively earned it significant merit, as it laid the groundwork for future improvements and set the stage for more advanced iterations. This approach underscores a vital lesson in development: sometimes, a good solution delivered quickly can be more valuable than a perfect one delivered too late.

The initial success provided a strong foundation upon which ESUPS could build. The insights gained from this deployment informed the development of a new version, which addresses the limitations of the first while enhancing user experience and data communication. The new dashboard takes these lessons to heart, incorporating more advanced features and a more intuitive design, ensuring that ESUPS continues to meet the needs of its partners more effectively. Now, let’s explore how this new version improves upon the original and further advances ESUPS's mission.

![](images/Disaster_Dashboard.png)

The newer version of the dashboard incorporates several effective design elements that help uninitiated users quickly grasp the context and understand the information presented. Building upon the foundation of the initial version, this iteration features a more logical layout, with distinct sections clearly labeled to make it easier for users to identify and comprehend different data categories at a glance. The use of diverse visualizations—such as a map, pie chart, bar chart, and line graph—enables multiple data points to be visualized simultaneously, offering a comprehensive overview of the situation.

A key improvement in this version is the effective use of white space. By strategically separating visual elements, the dashboard reduces clutter, ensuring that the information is not overwhelming and making it easier for users to digest. The clear representation of data ensures that each visualization has a specific purpose and is easy to interpret. For example, the Storage Facilities Map now clearly shows the geographical locations of storage facilities with available stock, providing important context. The pie chart uses distinct colors to differentiate between types of disasters, simplifying the interpretation of proportions, while the bar chart allows for a quick comparison of the number of people affected and the number of disasters by region. The line graph displays trends over time, with a dotted trend line to indicate the general direction of these trends.

The dashboard’s minimalist design elements further contribute to its effectiveness, an improvement from the earlier version. By avoiding unnecessary elements, the design minimizes distractions and enhances readability, ensuring that the focus remains on the data itself. Purposeful use of color now guides the viewer’s attention to key information more effectively. Interactive features, such as buttons labeled “SEE DISASTERS” and “SEE DETAILS,” suggest that users can engage more deeply with specific data points, which enhances both functionality and user engagement.

Moreover, the consistent color scheme and font style create a unified visual identity, lending a professional appearance and helping users navigate the information more intuitively. Altogether, these elements combine to present a cohesive design that significantly improves user experience and data comprehension compared to the earlier version.

However, as with any evolving tool, there is always room for further refinement. While this version addresses many of the initial challenges, the ongoing development process continues to identify areas where improvements can be made. By revisiting earlier designs, we can better understand how these refinements come about and how they contribute to a more effective and user-friendly dashboard. Let’s now take a look at one of the earlier pages to explore these opportunities for further enhancement.

![](images/Stock_Dashboard.png)

As the dashboard continues to evolve, ESUPS is taking the opportunity to further enhance its intuitiveness and user-friendliness. Building on the successes and lessons from earlier iterations, the latest redesign aims to address some of the shortcomings that were identified in the previous versions. By incorporating clear labels, units, and detailed explanations, the new design strives to provide users with the necessary context to interpret data effectively. For instance, sections like “NATIONAL AND COLLECTIVE ASSESSMENT OF STOCK QUANTITIES” and “TIME GAINED BY RE-DISPATCHING ITEMS BETWEEN LOCATIONS” will benefit from added clarity, helping users to understand the information presented more intuitively.

One of the key areas for improvement in this redesign is the refinement of the color scheme. The use of distinct and contrasting colors will make it easier for users to differentiate between categories in the bar charts. Additionally, clearly indicating what each color signifies, along with providing complete chart titles and fully displayed x-axis labels, will ensure that users can grasp the full context of the data being assessed. These visual enhancements are crucial for making the dashboard more accessible and easier to interpret, especially for users who may be less familiar with the underlying data.

Another critical improvement is the addition of context and explanations for key metrics. For example, providing details on figures like “39% - 14,615” under “National Stock Level Support” will remove ambiguity and highlight the relevance and importance of these numbers. Establishing clear connections between different data types—such as percentages, quantities, and time in minutes—will create a cohesive narrative that guides users through the data, making it easier to follow and more meaningful.

To further enhance the user experience, the redesigned dashboard will also focus on improving presentation aspects. Larger text sizes and decluttered visuals will enhance readability, allowing users to quickly grasp the information at a glance. Incorporating interactive elements, such as tooltips, will enable users to explore the data in greater depth and gain additional insights. Simplifying terminology and offering clear explanations for complex concepts, like “Time Gained by Re-Dispatching Items Between Locations,” will further aid in user understanding and ensure that the dashboard is accessible to a wider audience.

These iterative improvements are a testament to the dynamic nature of data visualization and communication within the industry. By continuously refining and enhancing the dashboard, ESUPS is committed to delivering a tool that not only meets but exceeds user needs. This ongoing process of iteration and enhancement underscores the organization’s dedication to creating a valuable resource for decision-making, ultimately helping ESUPS better serve its partners and fulfill its mission